In [1]:
import requests
from bs4 import BeautifulSoup
import re
from tqdm.notebook import tqdm

import pandas as pd

# 수집기간 정의

In [2]:
import datetime

In [3]:
days_range = []
start = datetime.datetime.strptime('2018.01.01', '%Y.%m.%d')
end = datetime.datetime.strptime('2018.12.31', '%Y.%m.%d')
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end - start).days)]

for date in date_generated:
    days_range.append(date.strftime('%Y.%m.%d'))

In [4]:
days_range[:3]

['2018.01.01', '2018.01.02', '2018.01.03']

# 하루 전체 기사 수

In [5]:
def get_daily_total_article_num(html):
    soup = BeautifulSoup(html.text, 'html.parser')
    daily_total_article_num = int(soup.find('div', attrs={'class': 'title_desc all_my'}).text.split('/')[1].strip()[:-1].replace(',', ''))
    return daily_total_article_num

# 제목과 네이버뉴스 주소 크롤링

In [6]:
def get_title_url(url, start_num):
    html = requests.get(url.format(query=query, start_date=day, end_date=day, start_date2=day2, end_date2=day2, start_num=start_num))
    soup = BeautifulSoup(html.text, 'html.parser')
    p = re.compile('sp_nws')
    news_list = soup.find_all('li', attrs={'id': p})
    
    title_list = []
    article_url_list = []
    source_list = []
    for idx, news in enumerate(news_list):
        if news.find('dd', attrs={'class': 'txt_inline'}).find('a', attrs={'class': '_sp_each_url'}):  # 네이버뉴스 유무
            title = news.find('dt').text
            article_url = news.find('dd', attrs={'class': 'txt_inline'}).find('a', attrs={'class': '_sp_each_url'}).get('href')
            source = news.find('dd', attrs={'class': 'txt_inline'}).find('span', attrs={'class': '_sp_each_source'}).text
            title_list.append(title)
            article_url_list.append(article_url)
            source_list.append(source)
            
    return title_list, article_url_list, source_list

# 본문 크롤링

In [7]:
def get_content(article_url_list):
    content_list = []
    for url in article_url_list:
        html = requests.get(url)
        soup = BeautifulSoup(html.text, 'html.parser')
        if soup.find('div', attrs={'id': 'newsEndContents'}):
            text = soup.find('div', attrs={'id': 'newsEndContents'}).text.strip()
            text = re.sub(' +', ' ', text)
            text = re.sub('\s', ' ', text)
            text = ' '.join(text.split())
        elif soup.find('div', attrs={'id': 'articleBodyContents'}):
            text = soup.find('div', attrs={'id': 'articleBodyContents'}).text.replace('\n', ' ')
            text = re.sub(' +', ' ', text)
            text = re.sub('\s', ' ', text)
            text = ' '.join(text.split())
            text = text.replace('// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}', '').strip()
        elif soup.find('div', attrs={'id': 'articeBody'}):
            text = soup.find('div', attrs={'id': 'articeBody'}).text.replace('\n', ' ').strip()
            text = re.sub(' +', ' ', text)
            text = re.sub('\s', ' ', text)
            text = ' '.join(text.split())
        else:
            print('또 다른 유형이 나왔습니다.')
        content_list.append(text)
        
    return content_list

# 2018년 전체 KLPGA 기사 크롤링

In [8]:
query = 'KLPGA'
url = '''https://search.naver.com/search.naver?&where=news&query={query}&sm=tab_pge&sort=1&photo=0&field=0&reporter_article=&pd=3
        &ds={start_date}&de={end_date}&docid=&nso=so:dd,p:from{start_date2}to{end_date2},a:all&mynews=0&start={start_num}&refresh_start=0'''
title = []
article_url = []
source = []
content = []

for day in tqdm(days_range):
    try:
        start_num = 1
        day2 = day.replace('.', '')
        html = requests.get(url.format(query=query, start_date=day, end_date=day, start_date2=day2, end_date2=day2, start_num=start_num))
        daily_total_article_num = get_daily_total_article_num(html)
        for start_num in range(daily_total_article_num // 10 + 1):
            title_list, article_url_list, source_list = get_title_url(url, start_num)
            content_list = get_content(article_url_list)
            title += title_list
            article_url += article_url_list
            source += source_list
            content += content_list
    except Exception as e:
        print(e)
        print(day)

In [9]:
len(title), len(content)

(27016, 27016)

In [10]:
data = {'url': url, 'title': title, 'source': source, 'content': content}

In [11]:
pd.DataFrame(data).head()

,url,title,source,content
0,https://search.naver.com/search.naver?&where=n...,"한국 낭자들, 이번에는 LPGA 15승 넘어설까",세계일보,2018시즌 역대 최다승 도전미국 골프전문 매체 골프채널이 선정한 지난해 미국여자프...
1,https://search.naver.com/search.naver?&where=n...,2018시즌 역대 최다승 합작 도전.. 코리안 군단의 위세는 계속된다,파이낸셜뉴스,작년 10명 총 15승 합작유소연.박성현.김인경 등 최다승 합작 달성여부 관심 LP...
2,https://search.naver.com/search.naver?&where=n...,LPGA 한국 자매들 당찬 ‘16승’ 출사표,서울신문,[서울신문]지난해 역대 최다승 타이기록인 15승(메이저대회 3승 포함)을 합작한 ‘...
3,https://search.naver.com/search.naver?&where=n...,"[데스크 칼럼] 새해, 실패에 대한 단상",부산일보,/김마선 교육팀장 새해가 또 밝았다. 무심한 시간에다 '마디'를 만들어 새 희망을 ...
4,https://search.naver.com/search.naver?&where=n...,"'황금개띠 골퍼' 전인지 ""새 후원사 KB금융 '천군만마'… 성장통 끝냈으니 훨...",한국경제,도전 2018! (5) 지난해 LPGA 준우승만 5번우승 못했지만 꾸준했던 한해허리...


In [12]:
article_2018 = pd.DataFrame(data)

# 2018년 전체 KLPGA 네이버기사 csv파일로 저장

In [13]:
article_2018.to_csv('data/klpga_2018_article.csv', encoding='utf-8-sig')